# VERSION 2.0

In [17]:
%matplotlib qt

In [18]:
# importing required libraries
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint as pp
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.linear_model import LinearRegression

# getting the battery data
bs_all = [
    'B0005', 'B0006', 'B0007', 'B0018', 'B0025', 'B0026', 'B0027', 'B0028', 'B0029', 'B0030', 'B0031', 'B0032', 
    'B0033', 'B0034', 'B0036', 'B0038', 'B0039', 'B0040', 'B0041', 'B0042', 'B0043', 'B0044', 'B0045', 'B0046', 
    'B0047', 'B0048', 'B0049', 'B0050', 'B0051', 'B0052', 'B0053', 'B0054', 'B0055', 'B0056',
]
ds = []
for b in bs_all:
    ds.append(loadmat(f'DATA/{b}.mat'))
    
types = []
times = []
ambient_temperatures = []
datas = []

for i in range(len(ds)):
    x = ds[i][bs_all[i]]["cycle"][0][0][0]
    ambient_temperatures.append(x['ambient_temperature'])
    types.append(x['type'])
    times.append(x['time'])
    datas.append(x['data'])

# clubbing all the compatible batteries together
# Batteries are compatible if they were recorded under similar conditions
# And their data size match up
bs_compt = {}

for i in range(len(bs_all)):
    sz = 0
    for j in range(datas[i].size):
        if types[i][j] == 'charge':
            sz += 1
    if bs_compt.get(sz):
        bs_compt[sz].append(bs_all[i])
    else: 
        bs_compt[sz] = [ bs_all[i] ]
pp(bs_compt)

{25: ['B0049', 'B0050', 'B0051', 'B0052'],
 31: ['B0025', 'B0026', 'B0027', 'B0028'],
 40: ['B0029', 'B0030', 'B0031', 'B0032'],
 47: ['B0038', 'B0039', 'B0040'],
 55: ['B0053'],
 67: ['B0041'],
 72: ['B0045', 'B0046', 'B0047', 'B0048'],
 102: ['B0054', 'B0055', 'B0056'],
 113: ['B0042', 'B0043', 'B0044'],
 134: ['B0018'],
 170: ['B0005', 'B0006', 'B0007'],
 197: ['B0033', 'B0034', 'B0036']}


In [19]:
## CRITICAL TIME POINTS FOR A CYCLE
## We will only these critical points for furthur training

## TEMPERATURE_MEASURED
## => Time at highest temperature

## VOLTAGE_MEASURED
## => Time at lowest Voltage

## VOLTAGE_LOAD
## => First time it drops below 1 volt after 1500 time

def getTemperatureMeasuredCritical(tm, time):
    high = 0
    critical = 0
    for i in range(len(tm)):
        if (tm[i] > high):
            high = tm[i]
            critical = time[i]
    return critical

def getVoltageMeasuredCritical(vm, time):
    low = 1e9
    critical = 0
    for i in range(len(vm)):
        if (vm[i] < low):
            low = vm[i]
            critical = time[i]
    return critical

def getVoltageLoadCritical(vl, time):
    for i in range(len(vl)):
        if (time[i] > 1500 and vl[i] < 1):
            return time[i]
    return -1

In [20]:
## Data Structure
## Cycles[battery][param][cycle]
## Cycles[battery][Capacity][cycle]


Cycles = {}
params = ['Temperature_measured', 'Voltage_measured', 'Voltage_load', 'Time']

# iterate over all the battery sets
for bs_cmpt in bs_compt:
    # getting data for a given set
    bs = bs_compt[bs_cmpt]
    for i in range(len(bs)):
        Cycles[bs[i]] = {}
        for param in params:
            Cycles[bs[i]][param] = []
            for j in range(datas[i].size):
                if types[i][j] == 'discharge':
                    Cycles[bs[i]][param].append(datas[i][j][param][0][0][0])

        cap = []
        for j in range(datas[i].size):
            if types[i][j] == 'discharge':
                cap.append(datas[i][j]['Capacity'][0][0][0][0])
        Cycles[bs[i]]['Capacity'] = np.array(cap)
        Cycles[bs[i]]['count'] = len(Cycles[bs[0]]['Capacity'])
        
    # preparing data for regression model
    temperature_measured = []
    voltage_measured = []
    voltage_load = []
    capacity = []

    for b in bs:
        for c in Cycles[b]['Capacity']:
            capacity.append(c)

    for _ in range(len(bs)):
        for i in range(Cycles[bs[_]]['count']):
            temperature_measured.append(getTemperatureMeasuredCritical(Cycles[bs[_]]['Temperature_measured'][i], Cycles[bs[_]]['Time'][i]))
            voltage_measured.append(getVoltageMeasuredCritical(Cycles[bs[_]]['Voltage_measured'][i], Cycles[bs[_]]['Time'][i]))
            voltage_load.append(getVoltageLoadCritical(Cycles[bs[_]]['Voltage_load'][i], Cycles[bs[_]]['Time'][i]))
    
    # creating the model
    X = []
    for i in range(len(temperature_measured)):
        X.append(np.array([temperature_measured[i], voltage_measured[i], voltage_load[i]]))
    X = np.array(X)
    y = np.array(capacity)

    # creating train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

    # fitting the model
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    # test
    y_pred = regressor.predict(X_test)
    
    # evaluating the model
    diff = 0
    total = 0
    for i in range(len(y_test)):
        diff += abs(y_test[i] - y_pred[i])
        total += y_test[i]
    diff /= len(y_test)
    total /= len(y_test)
    accuracy = ((total - diff) / total) * 100
    print(f'Battery set: ' + str(bs))
    print(f'Average Difference Between Predicted and Real Capacities: {diff}')
    print(f'Accuracy: {accuracy}\n')
    

Battery set: ['B0005', 'B0006', 'B0007']
Average Difference Between Predicted and Real Capacities: 0.013404734961093144
Accuracy: 99.15812921489568

Battery set: ['B0018']
Average Difference Between Predicted and Real Capacities: 0.0001504458509620327
Accuracy: 99.9903768373228



IndexError: list index out of range